# Preprocessing

Steps involved:
* Convert string values from text file to float64
* Replace NaN with zero

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno

%matplotlib inline

In [57]:
# The filtered data contains NaN representing suppressed p-values.
#path_to_file = './../data/emQTL_CorrelationMatrix_significant_pvalues.txt'
path_to_file = './../data/emQTL_CorrelationMatrix_significant_PCCs.txt'
data_raw = pd.read_csv(path_to_file, delim_whitespace=True, index_col=0)

In [58]:
# Convert <str> values to <float64>
data = data_raw.infer_objects()

In [59]:
set(data.dtypes)

{dtype('float64')}

In [60]:
data.head()

,TNFRSF4,MXRA8,ATAD3A,MMP23B,SLC35E2,MORN1,KCNAB2,KLHL21,TNFRSF9,ENO1,...,ATP11C,CXorf40B,MAGEA4,MAGEA12,CETN2,PNMA5,PNMA3,TREX2,ARHGAP4,FLNA
cg00002224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00002426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00002593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00002719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cg00003287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
data.shape

(27561, 2664)

In [62]:
data.fillna(0, inplace=True)

In [63]:
data.describe()

,TNFRSF4,MXRA8,ATAD3A,MMP23B,SLC35E2,MORN1,KCNAB2,KLHL21,TNFRSF9,ENO1,...,ATP11C,CXorf40B,MAGEA4,MAGEA12,CETN2,PNMA5,PNMA3,TREX2,ARHGAP4,FLNA
count,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,...,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000,27561.000000
mean,0.001020,-0.000363,-0.000178,-0.000723,-0.000245,-0.000022,-0.000245,-0.000247,-0.008528,-0.000112,...,0.002527,-0.000048,-0.000046,-0.000069,0.003197,0.000021,0.000175,-0.000044,-0.001725,-0.000022
std,0.037167,0.015077,0.010421,0.021222,0.012274,0.003711,0.012242,0.012360,0.175341,0.013396,...,0.054589,0.010569,0.005405,0.006573,0.053127,0.013412,0.018947,0.005180,0.040488,0.003714
min,-0.640119,-0.685457,-0.622229,-0.654785,-0.643821,-0.616101,-0.632275,-0.641874,-0.714242,-0.635664,...,-0.666624,-0.668961,-0.652988,-0.641353,-0.683733,-0.637267,-0.667089,-0.611410,-0.652898,-0.616646
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.668528,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.720029,0.634879,...,0.688650,0.615330,0.000000,0.000000,0.681397,0.632055,0.638440,0.000000,0.637303,0.000000


In [64]:
#path_to_disk = './../data/significant_pvalues_prep.csv'
path_to_disk = './../data/./../data/significant_pcc_prep.csv'
data.to_csv(path_to_disk, sep=',', columns=list(data.columns))
